<a href="https://colab.research.google.com/github/mnassar/py2vpy3xai/blob/main/python2vspython3XAIScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gitpython

In [ ]:
!pip install 3to2

In [ ]:
import requests

def search_github_repositories(query, repo_type='repositories', result_limit=10):
    url = f'https://api.github.com/search/{repo_type}?q={query}&per_page={result_limit}'
    print(url)
    response = requests.get(url, headers={'Accept': 'application/vnd.github.v3+json'})

    if response.status_code == 200:
        data = response.json()
        repositories = data.get('items', [])
        return repositories
    else:
        print(f"Error: {response.status_code}")
        return None

search_query = 'python3 programs'
result_limit = 100

# Search for repositories
repositories = search_github_repositories(search_query, result_limit=result_limit)
len( repositories )



https://api.github.com/search/repositories?q=python3 programs&per_page=100


100

In [ ]:
mkdir repos

mkdir: cannot create directory ‘repos’: File exists


In [ ]:
from git import Repo, remote
import os



for repo in repositories:
  try:
    repo_url = repo['clone_url']
    Repo.clone_from(repo_url, os.path.join("repos", repo['name']))
  except  Exception as e:
    print(e)
    continue

In [ ]:
!ls -l repos | wc -l

95


In [ ]:
import os
import chardet



def valid(file_path):
  # if it doesn't compile, it is not valid!
  if os.system(f"python3 -m py_compile {file_path}") != 0:
    return False

  # @ToDo: Files that are valid py3 and can't be converted to Py2
  # should be included and labeled as py3
  if os.system(f"3to2 {file_path}") != 0:
    return False


  with open(file_path, 'rb') as py_file:
    source_code = py_file.read()
    # Detect the encoding of the file
    result = chardet.detect(source_code)
    encoding = result['encoding']
    # print(encoding)
    # Read the file using the detected encoding
    if encoding == "utf-8" or encoding =="ascii":
      return True
      # if b'match' in source_code and b'case' in source_code:
      #   return False
      # else:
      #   return True
    else:
      return False

# valid("example2.py")

In [ ]:
# !3to2 example2.py

In [ ]:
import chardet

file_path = "/content/repos/CLI-in-Python/src/console/Console.py"
output_file = "python3.py"


def read_file(file_path, output, require_valid=True):
    line_count = 0
    v = False
    if require_valid:
      v = valid(file_path)
    if (not require_valid) or ( require_valid and v ):
      with open(file_path, 'r') as py_file:
          lines = py_file.readlines()
          for line in lines:
              if line.strip() != "" and not line.strip().startswith("#"):  # Skip writing blank and comment lines
                  output.write(line)
                  line_count += 1

    return v, line_count

with open(output_file, 'w', encoding='utf-8') as output:
  line_count = read_file(file_path, output)
print (line_count)


(True, 248)


In [ ]:
import chardet

def download_python_files_from_repositories(repos, output_file, max_lines=100000):
    line_count = 0
    with open(output_file, 'w', encoding='utf-8') as output:
        for root, dirs, files in os.walk(repos):
            for file_name in files:
                if line_count >= max_lines:
                  break
                if file_name.endswith('.py'):
                    file_path = os.path.join(root, file_name)
                    try:
                        valid, file_line_count = read_file(file_path, output)
                        if not valid:
                          invalid_files.append(file_path)
                        if file_line_count > 0:
                          line_count += file_line_count
                          print (line_count)
                        output.write("\n")
                    except Exception as e:
                        print(f"Error reading file {file_path}: {e}")
                        continue

invalid_files = []
download_python_files_from_repositories("repos", "python3.py", 10000) # 100 * 1000

In [ ]:
print (len(invalid_files))

94


In [ ]:
!wc -l python3.py

10824 python3.py


In [ ]:
!cp python3.py python2.py

In [ ]:
!3to2 python2.py -w

In [ ]:
!wc -l python2.py

10960 python2.py


In [ ]:
def clean_multi_line_comments(out_file, in_file):
  with open(out_file, 'w', encoding='utf-8') as output:
    with open(in_file, 'r') as py_file:
      comment = False
      for line in py_file:
        if line.strip().count('"""') % 2 == 1:
          # print(1, line)
          comment =  not comment # toggle comment
          continue
        if '"""' in line.strip(): # count is even in this case, just ignore line
          # print(2, line)
          continue
        if not comment and line.strip() != "":
          output.write(line)
        # else:
          # print(3, line)

In [ ]:
# clean multi line comments from python2 and python3 overall files
clean_multi_line_comments("python3_clean.py", "python3.py")
clean_multi_line_comments("python2_clean.py", "python2.py")


In [ ]:
!wc -l python2_clean.py

10135 python2_clean.py


In [ ]:
!wc -l python3_clean.py

9999 python3_clean.py


In [ ]:
# divide in three files
!sort python2_clean.py | uniq > python2sorted.py
!sort python3_clean.py | uniq > python3sorted.py
!comm -1 -2 python2sorted.py python3sorted.py > common.py
!comm -3 -1 python2sorted.py python3sorted.py > python3only.py
!comm -3 -2 python2sorted.py python3sorted.py > python2only.py

In [ ]:
!wc -l python2sorted.py

6535 python2sorted.py


In [ ]:
!wc -l python2_clean.py

10135 python2_clean.py


In [ ]:
!wc -l python3only.py

2758 python3only.py


In [ ]:
!wc -l python2only.py

2822 python2only.py


In [ ]:
!wc -l common.py

3713 common.py


In [ ]:
# add invalid files to python3
lc = 0
with open("python3onlyprime.py", 'w', encoding='utf-8') as output:
  for file_path in invalid_files:
    if os.system(f"python3 -m py_compile {file_path}") == 0:
      _, lc_file = read_file(file_path, output, require_valid=False)
      lc += lc_file
      print(file_path, lc)

print (lc)


In [ ]:
!wc -l python3onlyprime.py

664 python3onlyprime.py


In [ ]:
clean_multi_line_comments("python3onlyprime_clean.py", "python3onlyprime.py")

In [ ]:
!wc -l python3onlyprime_clean.py

607 python3onlyprime_clean.py


In [ ]:
!sort python3onlyprime_clean.py | uniq > python3onlyprime_clean_sorted.py

In [ ]:
# add these lines to python3
!cat python3onlyprime_clean_sorted.py >> python3only.py

In [ ]:
!wc -l python3only.py

3283 python3only.py


In [ ]:
# line of code labeling
import pandas as pd

with open("common.py", 'r') as f:
  # we assume common are py3 but they could have a class of their own
  df1 = pd.DataFrame({"lines of code": [line.strip() for line in set(f.readlines())], "class": 3})

with open("python2only.py", 'r') as f:
  df2 = pd.DataFrame({"lines of code": [line.strip() for line in set(f.readlines())], "class": 2})

with open("python3only.py", 'r') as f:
  df3 = pd.DataFrame({"lines of code": [line.strip() for line in set(f.readlines())], "class": 3})

df = pd.concat([df1, df2, df3], ignore_index=True)

df[df["class"]==3]

,lines of code,class
0,from llvmlite import ir,3
1,if global_mem > 0 then reply else message counter,3
2,sensor_type = struct.unpack(,3
3,delta=date_list[j]-date_list[i],3
4,"variable=self.time_delta_max_tmp,",3
...,...,...
9808,"raise NoEV3('multiple EV3 found, you need to s...",3
9809,pygments_style = 'sphinx',3
9810,discontinued = controller.remove_ephemeral_hid...,3
9811,"pcap_file = (""/Users/bwarner/PycharmProjects/C...",3


In [ ]:
df1

,lines of code,class
0,from llvmlite import ir,3
1,if global_mem > 0 then reply else message counter,3
2,sensor_type = struct.unpack(,3
3,delta=date_list[j]-date_list[i],3
4,"variable=self.time_delta_max_tmp,",3
...,...,...
3708,def test_opError():,3
3709,self._device = hid.device(),3
3710,"LVX(16), # DESTINATION - step1 + step3 (DATA32)",3
3711,return self._commands,3


In [ ]:
df[df["class"]==2]

,lines of code,class
3713,WARNING_MEMORY = '\x08',2
3714,TRUNC = '\x15' # !< Truncate r = (float)((int...,2
3715,"assert isinstance(value, int), u""volume needs ...",2
3716,"motor_pos, **_3to2kwargs",2
3717,u'sphinx.ext.viewcode',2
...,...,...
6530,"results = censys.ipv4.search(f""{fingerprint}"",...",2
6531,"response = super(Session, self).request(method...",2
6532,"LCS(path + u'.rsf'),",2
6533,"QMessageBox.about(self, u""Info"", u""File was sa...",2


In [ ]:
df

,lines of code,class
0,from llvmlite import ir,3
1,if global_mem > 0 then reply else message counter,3
2,sensor_type = struct.unpack(,3
3,delta=date_list[j]-date_list[i],3
4,"variable=self.time_delta_max_tmp,",3
...,...,...
9808,"raise NoEV3('multiple EV3 found, you need to s...",3
9809,pygments_style = 'sphinx',3
9810,discontinued = controller.remove_ephemeral_hid...,3
9811,"pcap_file = (""/Users/bwarner/PycharmProjects/C...",3


In [ ]:
# add explainability annotations
import re
patterns = {
        #py3/ print -- 0 or more space -- ( -- 0 or more space -- (1 or more no space)
        "print(": re.compile(r"\bprint\s*\(\s*\S+"),
        #py2 (or early versions of py3)/ __future__ -- one or more space
        "__future__": re.compile(r"\b__future__\s+"),
        #py2/ xrange -- 0 or more space -- (
        "xrange": re.compile(r"\bxrange\s*\("),
        #py3/ space -- range -- 0 or more spaces -- (
        " range": re.compile(r"\brange\s*\("),
        #py2/ u (for unicode) -- ' or " -- 0 or more spaces -- 1 or more no spaces
        'u"': re.compile(r'\bu[\'|\"]\s*\S+'),
        #print -- 1 or more space -- at least one non-parenthesis -- one or more non space
        "print ": re.compile(r"\bprint\s+[^\(]+\S+"),
        #py2/ boundary -- unicode -- (
        "unicode(": re.compile(r"\bunicode\s*\("),
        #py3/ space -- __next__ -- space -- (
        "__next__(": re.compile(r"\b__next__\s*\("),
        #py2/ raw_input
        "raw_input(": re.compile(r"\braw_input\s*\(")
    }
count_patterns = { k:0 for k in patterns.keys()}

def extract_matches(key, pattern, text):
  # print(text)
  match = re.search(pattern, text)
  if match:
    count_patterns[key] += 1
    return match.group()
  else:
    return None

for key, pattern in patterns.items():
  df[key] = df["lines of code"].map(lambda x: extract_matches(key, pattern, x))

# for key in patterns:
#   print (key)
#   print ( df[df[key].notnull()][key] )
#   print( )


In [ ]:
count_patterns

{'print(': 208,
 '__future__': 3,
 'xrange': 30,
 ' range': 32,
 'u"': 1570,
 'print ': 77,
 'unicode(': 39,
 '__next__(': 0,
 'raw_input(': 6}

In [ ]:
# combine explanations
def exp_concat(s1, s2):
  if s1 == None:
    return s2
  elif s2 == None:
    return s1
  else:
    return s1 + " " + s2

df["explanation"]=None
for key in patterns:
  df["explanation"] = df["explanation"].combine(df[key], exp_concat)
  # df = df.drop(labels=[key], axis=1)


# df["explanation"]

In [ ]:
df

,lines of code,class,print(,__future__,xrange,range,"u""",print,unicode(,__next__(,raw_input(,explanation
0,from llvmlite import ir,3,None,None,None,None,None,None,None,None,None,None
1,if global_mem > 0 then reply else message counter,3,None,None,None,None,None,None,None,None,None,None
2,sensor_type = struct.unpack(,3,None,None,None,None,None,None,None,None,None,None
3,delta=date_list[j]-date_list[i],3,None,None,None,None,None,None,None,None,None,None
4,"variable=self.time_delta_max_tmp,",3,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9808,"raise NoEV3('multiple EV3 found, you need to s...",3,None,None,None,None,None,None,None,None,None,None
9809,pygments_style = 'sphinx',3,None,None,None,None,None,None,None,None,None,None
9810,discontinued = controller.remove_ephemeral_hid...,3,None,None,None,None,None,None,None,None,None,None
9811,"pcap_file = (""/Users/bwarner/PycharmProjects/C...",3,None,None,None,None,None,None,None,None,None,None


In [ ]:
df[df["class"]==2][df["explanation"].notnull()]

<ipython-input-41-ef4fbb1c18f1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df["class"]==2][df["explanation"].notnull()]


,lines of code,class,print(,__future__,xrange,range,"u""",print,unicode(,__next__(,raw_input(,explanation
3715,"assert isinstance(value, int), u""volume needs ...",2,None,None,None,None,"u""volume",None,None,None,None,"u""volume"
3717,u'sphinx.ext.viewcode',2,None,None,None,None,u'sphinx.ext.viewcode',None,None,None,None,u'sphinx.ext.viewcode'
3718,"winIcon = resource_path(os.path.join(u'src', w...",2,None,None,None,None,"u'src',",None,None,None,None,"u'src',"
3719,target_pos_outer = cur_mov[u'target_motor_pos'...,2,None,None,None,None,u'target_motor_pos'][1],None,None,None,None,u'target_motor_pos'][1]
3720,self._current_movement[u'started_at'],2,None,None,None,None,u'started_at'],None,None,None,None,u'started_at']
...,...,...,...,...,...,...,...,...,...,...,...,...
6528,"assert value in (1, -1), u""allowed polarity_ri...",2,None,None,None,None,"u""allowed",None,None,None,None,"u""allowed"
6529,"u'relations.html', # needs 'show_related': Tr...",2,None,None,None,None,"u'relations.html',",None,None,None,None,"u'relations.html',"
6530,"results = censys.ipv4.search(f""{fingerprint}"",...",2,None,None,None,None,"u""443.https.tls.certificate.parsed.extensions....",None,None,None,None,"u""443.https.tls.certificate.parsed.extensions...."
6532,"LCS(path + u'.rsf'),",2,None,None,None,None,"u'.rsf'),",None,None,None,None,"u'.rsf'),"


In [ ]:
# download the dataset as csv file
df.to_csv("dataset10k.csv", index=False)

In [ ]:
!wc -l dataset10k.csv

9814 dataset10k.csv
